In [1]:
import torch
from PIL import Image
from diffusers import AutoencoderKL, UNet2DModel, DDIMPipeline, DDIMScheduler, DDPMPipeline, DDPMScheduler
from progressive_distillation import DistillationPipelineOld
from diffusers.optimization import get_scheduler
from diffusers.training_utils import EMAModel
import math
import requests
from torchvision.transforms import (
    CenterCrop,
    Compose,
    InterpolationMode,
    Normalize,
    RandomHorizontalFlip,
    Resize,
    ToTensor,
    ToPILImage
)
from torch.utils.data import Dataset
from accelerate import Accelerator
from progressive_distillation import utils
from tqdm import tqdm
import torch.nn.functional as F
import os

In [2]:
os.environ['CUDA_LAUNCH_BLOCKING']="1"
os.environ['TORCH_USE_CUDA_DSA']="1"

# # Enable/Disable CUDA
torch.backends.cudnn.enabled = True
torch.cuda.is_available = lambda : True

In [3]:
torch.manual_seed(0)
training_config = utils.DiffusionTrainingArgs()

In [4]:
# Load an image of my dog for this example

image_url = "https://i.imgur.com/IJcs4Aa.jpeg"
image = Image.open(requests.get(image_url, stream=True).raw)

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7f329fa17650>

In [ ]:
# Define the transforms to apply to the image for training
augmentations = utils.get_train_transforms(training_config)

In [ ]:
class SingleImageDataset(Dataset):
    def __init__(self, image, batch_size):
        self.image = image
        self.batch_size = batch_size

    def __len__(self):
        return self.batch_size

    def __getitem__(self, idx):
        return self.image

In [ ]:
train_image = augmentations(image.convert("RGB"))
train_dataset = SingleImageDataset(train_image, training_config.batch_size)

In [ ]:
teacher = UNet2DModel.from_pretrained("bglick13/minnie-diffusion")
distiller = DistillationPipelineOld()

In [ ]:
N = 1000
generator = torch.manual_seed(0)

In [5]:
teacher = UNet2DModel.from_pretrained("bglick13/minnie-diffusion")
N = 1000
distilled_images = []
for distill_step in range(2):
    print(f"Distill step {distill_step} from {N} -> {N // 2}")
    teacher, distilled_ema, distill_accelrator = distiller(teacher, N, train_dataset, epochs=300, batch_size=training_config.batch_size)
    N = N // 2
    new_scheduler = DDPMScheduler(num_train_timesteps=N, beta_schedule="squaredcos_cap_v2")
    pipeline = DDPMPipeline(
        unet=distill_accelrator.unwrap_model(distilled_ema.averaged_model if training_config.use_ema else teacher),
        scheduler=new_scheduler,
    )

    # run pipeline in inference (sample random noise and denoise)
    images = pipeline(generator=generator, batch_size=training_config.batch_size, output_type="numpy").images

    # denormalize the images and save to tensorboard
    images_processed = (images * 255).round().astype("uint8")
    distilled_images.append(images_processed[0])

Distill step 0 from 1000 -> 500


NameError: name 'distiller' is not defined

In [6]:
# Display train image for reference
train_image_display = train_image * 0.5 + 0.5
train_image_display = ToPILImage()(train_image_display)
display(train_image_display)

for i, image in enumerate(distilled_images):
    print(f"Distilled image {i}")
    display(Image.fromarray(image))
    Image.fromarray(image).save(f"distilled_{i}.png")

NameError: name 'train_image' is not defined

In [ ]:
display(Image.fromarray(images_processed[0]))
display(Image.fromarray(images_processed[1]))

In [11]:
from diffusers import DiffusionPipeline
import torch

import torch
from diffusers import DiffusionPipeline
from diffusers import DPMSolverMultistepScheduler
from torch import Generator


# path = 'segmind/portrait-finetuned' # Path to the appropriate model-type
path = 'CompVis/stable-diffusion-v1-4'
# path = 'segmind/small-sd'
# Insert your prompt below.
# prompt = "Faceshot Portrait of pretty young (18-year-old) Caucasian wearing a high neck sweater, (masterpiece, extremely detailed skin, photorealistic, heavy shadow, dramatic and cinematic lighting, key light, fill light), sharp focus, BREAK epicrealism"
# # Insert negative prompt below. We recommend using this negative prompt for best results.
# negative_prompt = "(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck" 
# prompt = "a woman in a red and gold costume with feathers on her head"
# extra_prompt = ", facing the camera, photograph, highly detailed face, depth of field, moody light, style by Yasmin Albatoul, Harry Fayt, centered, extremely detailed, Nikon D850, award winning photography"
# negative_prompt = "cartoon, anime, ugly, (aged, white beard, black skin, wrinkle:1.1), (bad proportions, unnatural feature, incongruous feature:1.4), (blurry, un-sharp, fuzzy, un-detailed skin:1.2), (facial contortion, poorly drawn face, deformed iris, deformed pupils:1.3), (mutated hands and fingers:1.5), disconnected hands, disconnected limbs"
prompt = "Living in the light with a businesswoman"

torch.set_grad_enabled(False)
torch.backends.cudnn.benchmark = True

# Below code will run on gpu, please pass cpu everywhere as the device and set 'dtype' to torch.float32 for cpu inference.
with torch.inference_mode():
    gen = Generator("cuda")
    gen.manual_seed(1674753452)
    pipe = DiffusionPipeline.from_pretrained(path, torch_dtype=torch.float16, safety_checker=None, requires_safety_checker=False)
    pipe.to('cuda')
    pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
    pipe.unet.to(device='cuda', dtype=torch.float16, memory_format=torch.channels_last)

    img = pipe(prompt=prompt, width=512, height=512, num_inference_steps=25, guidance_scale = 7, num_images_per_prompt=1, generator = gen).images[0]
    img.save("businesswoman.png")

RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
